In [41]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold
import warnings
import xgboost as xgb

In [17]:
data = pd.read_csv('./MachineLearningCVE/Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv', skipinitialspace=True)

In [21]:
data.head()

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,0,15/02/2018 08:25:18,112641158,3,0,0,0,0,0,...,0,0.0,0.000000,0,0,56320579.0,7.042784e+02,56321077,56320081,Benign
1,22,6,15/02/2018 08:29:05,37366762,14,12,2168,2993,712,0,...,32,1024353.0,649038.754495,1601183,321569,11431221.0,3.644991e+06,15617415,8960247,Benign
2,47514,6,15/02/2018 08:29:42,543,2,0,64,0,64,0,...,32,0.0,0.000000,0,0,0.0,0.000000e+00,0,0,Benign
3,0,0,15/02/2018 08:28:07,112640703,3,0,0,0,0,0,...,0,0.0,0.000000,0,0,56320351.5,3.669884e+02,56320611,56320092,Benign
4,0,0,15/02/2018 08:30:56,112640874,3,0,0,0,0,0,...,0,0.0,0.000000,0,0,56320437.0,7.198347e+02,56320946,56319928,Benign


In [22]:
data.columns

Index(['Dst Port', 'Protocol', 'Timestamp', 'Flow Duration', 'Tot Fwd Pkts',
       'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
       'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
       'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
       'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags',
       'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s',
       'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean',
       'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt',
       'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt',
       'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg',
      

In [25]:
data['Dst Port'].nunique()

21732

In [26]:
data['Protocol'].unique()

array([ 0,  6, 17], dtype=int64)

In [27]:
data['Protocol'].value_counts()

6     684486
17    345524
0      18565
Name: Protocol, dtype: int64

In [24]:
data['Label'].unique()

array(['Benign', 'DoS attacks-GoldenEye', 'DoS attacks-Slowloris'],
      dtype=object)

In [31]:
data['Label'].value_counts()

Benign                   996077
DoS attacks-GoldenEye     41508
DoS attacks-Slowloris     10990
Name: Label, dtype: int64

In [36]:
data.replace({"Label": {'Benign': 0, 'DoS attacks-GoldenEye': 1, "DoS attacks-Slowloris": 1}}, inplace=True)

In [37]:
data

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,0,15/02/2018 08:25:18,112641158,3,0,0,0,0,0,...,0,0.0,0.000000,0,0,56320579.0,7.042784e+02,56321077,56320081,0
1,22,6,15/02/2018 08:29:05,37366762,14,12,2168,2993,712,0,...,32,1024353.0,649038.754495,1601183,321569,11431221.0,3.644991e+06,15617415,8960247,0
2,47514,6,15/02/2018 08:29:42,543,2,0,64,0,64,0,...,32,0.0,0.000000,0,0,0.0,0.000000e+00,0,0,0
3,0,0,15/02/2018 08:28:07,112640703,3,0,0,0,0,0,...,0,0.0,0.000000,0,0,56320351.5,3.669884e+02,56320611,56320092,0
4,0,0,15/02/2018 08:30:56,112640874,3,0,0,0,0,0,...,0,0.0,0.000000,0,0,56320437.0,7.198347e+02,56320946,56319928,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,50111,6,15/02/2018 09:04:42,22,3,0,31,0,31,0,...,20,0.0,0.000000,0,0,0.0,0.000000e+00,0,0,0
1048571,443,6,15/02/2018 09:03:55,54682783,5,1,123,46,46,0,...,20,158783.0,0.000000,158783,158783,54523813.0,0.000000e+00,54523813,54523813,0
1048572,443,6,15/02/2018 09:03:56,53682093,5,1,123,46,46,0,...,20,259719.0,0.000000,259719,259719,53421756.0,0.000000e+00,53421756,53421756,0
1048573,443,6,15/02/2018 09:03:55,54683364,5,1,123,46,46,0,...,20,158870.0,0.000000,158870,158870,54523593.0,0.000000e+00,54523593,54523593,0


In [38]:
features = [f for f in data.columns if f not in ['Timestamp', 'Label']]

In [39]:
x_train = data[features]
y_train = data['Label']

In [43]:
folds = 5
seed = 2020

In [44]:
# use StratifiedKFold instead of KFold to solve unbalanced datasets
kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)

In [45]:
train = np.zeros(x_train.shape[0])

In [48]:
def cv_model(clf, x_train, y_train, clf_name):
    '''
        @param clf: 分类器包名(lgb&xgb)、分类器实例名(CatBoostClassifier)
        @param x_train: 训练集X
        @param y_train: 训练集y
        @param clf_name: 分类器名：'lgb' 'xgb' 'cat'

        @return: model 最后一次交叉训练后的模型
        @return: train 分类器在训练集上的预测结果
    '''

    folds = 5
    seed = 2020
    # use StratifiedKFold instead of KFold to solve unbalanced datasets
    kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)

    train = np.zeros(x_train.shape[0])

    cv_accuracy_scores = []
    cv_f1_scores = []
    cv_auc_scores = []

    for i, (train_index, valid_index) in enumerate(kf.split(x_train, y_train)):
        print('************************************ {0}/{1} ************************************'.format(str(i+1), folds))
        trn_x, trn_y, val_x, val_y = x_train.iloc[train_index], y_train[train_index], x_train.iloc[valid_index], y_train[valid_index]
        
        if clf_name == "lgb":
            train_matrix = clf.Dataset(trn_x, label=trn_y)
            valid_matrix = clf.Dataset(val_x, label=val_y)

            params = {
                'boosting_type': 'gbdt',
                'objective': 'binary',
                'metric': 'auc',
                'min_child_weight': 5,
                'num_leaves': 2 ** 5,
                'lambda_l2': 10,
                'feature_fraction': 0.8,
                'bagging_fraction': 0.8,
                'bagging_freq': 4,
                'learning_rate': 0.1,
                'seed': 2020,
                'nthread': 28,
                'n_jobs':24,
                'silent': True,
                'verbose': -1,
                'is_unbalance': True, # unblanced datasets
            }

            model = clf.train(params, train_matrix, 50000, valid_sets=[train_matrix, valid_matrix], 
                              verbose_eval=200, early_stopping_rounds=200)
            val_score = model.predict(val_x, num_iteration=model.best_iteration)
            
            # 输出前20特征重要性从高到低
            print(list(sorted(zip(model.feature_name(), model.feature_importance("gain")), key=lambda x: x[1], reverse=True))[:20])
                
        if clf_name == "xgb":
            train_matrix = clf.DMatrix(trn_x , label=trn_y)
            valid_matrix = clf.DMatrix(val_x , label=val_y)
            
            params = {'booster': 'gbtree',
                      'objective': 'binary:logistic',
                      'eval_metric': 'auc',
                      'gamma': 1,
                      'min_child_weight': 1.5,
                      'max_depth': 5,
                      'lambda': 10,
                      'subsample': 0.7,
                      'colsample_bytree': 0.7,
                      'colsample_bylevel': 0.7,
                      'eta': 0.04,
                      'tree_method': 'exact',
                      'seed': 2020,
                      'nthread': 36,
                      "silent": True,
                      'is_unbalance': True, # unblanced datasets
                      }
            
            watchlist = [(train_matrix, 'train'),(valid_matrix, 'eval')]
            
            model = clf.train(params, train_matrix, num_boost_round=50000, evals=watchlist, verbose_eval=200, early_stopping_rounds=200)
            val_score  = model.predict(valid_matrix, ntree_limit=model.best_ntree_limit)

        # 交叉验证每次得到一部分结果
        val_pred = np.int8(np.round(val_score))
        train[valid_index] = val_pred
        
        cv_accuracy_scores.append(accuracy_score(val_y, val_pred))
        cv_f1_scores.append(f1_score(val_y, val_pred))
        cv_auc_scores.append(roc_auc_score(val_y, val_score))
        
        print(cv_accuracy_scores)
        print(cv_f1_scores)
        print(cv_auc_scores)
       
    print("%s_accuracy_score_list:" % clf_name, cv_accuracy_scores)
    print("%s_accuracy_score_mean:" % clf_name, np.mean(cv_accuracy_scores))
    print("%s_accuracy_score_std:" % clf_name, np.std(cv_accuracy_scores))
    
    print("%s_f1_score_list:" % clf_name, cv_f1_scores)
    print("%s_f1_score_mean:" % clf_name, np.mean(cv_f1_scores))
    print("%s_f1_score_std:" % clf_name, np.std(cv_f1_scores))
    
    print("%s_auc_score_list:" % clf_name, cv_auc_scores)
    print("%s_auc_score_mean:" % clf_name, np.mean(cv_auc_scores))
    print("%s_auc_score_std:" % clf_name, np.std(cv_auc_scores))
    
    return model, train

In [50]:
def lgb_model(x_train, y_train):
    lgb_trained_model, lgb_train = cv_model(lgb, x_train, y_train, "lgb")
    return lgb_trained_model, lgb_train
def xgb_model(x_train, y_train):
    xgb_trained_model, xgb_train = cv_model(xgb, x_train, y_train, "xgb")
    return xgb_trained_model, xgb_train


In [51]:
lgb_trained_model, lgb_train = lgb_model(x_train, y_train)

************************************ 1/5 ************************************


D:\Users\Forsaken\anaconda3\envs\ep2\lib\site-packages\lightgbm\basic.py:1224: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  .format(key))


[LightGBM] [Warning] num_threads is set with nthread=28, will be overridden by n_jobs=24. Current value: num_threads=24
[LightGBM] [Warning] Unknown parameter: silent
Training until validation scores don't improve for 200 rounds
[200]	training's auc: 1	valid_1's auc: 1
[400]	training's auc: 1	valid_1's auc: 1
[600]	training's auc: 1	valid_1's auc: 1
Early stopping, best iteration is:
[474]	training's auc: 1	valid_1's auc: 1
[('Fwd_Seg_Size_Min', 8459601.308177263), ('Dst_Port', 393009.21240997314), ('Init_Fwd_Win_Byts', 28216.482365053147), ('FIN_Flag_Cnt', 27764.225456006825), ('Flow_Duration', 12150.498362155631), ('Tot_Bwd_Pkts', 9494.1884361431), ('Fwd_Pkts/s', 6818.489770870656), ('Flow_IAT_Max', 6087.032292471267), ('Fwd_IAT_Min', 5686.24190192204), ('Fwd_Header_Len', 3865.825444865972), ('ACK_Flag_Cnt', 3464.4878742694855), ('Flow_IAT_Min', 1856.6076587520074), ('Fwd_IAT_Mean', 1785.2855235263705), ('Idle_Mean', 1338.0699462890625), ('Flow_Pkts/s', 1252.6954650469124), ('Fwd_IAT

D:\Users\Forsaken\anaconda3\envs\ep2\lib\site-packages\lightgbm\basic.py:1224: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  .format(key))


[LightGBM] [Warning] num_threads is set with nthread=28, will be overridden by n_jobs=24. Current value: num_threads=24
[LightGBM] [Warning] Unknown parameter: silent
Training until validation scores don't improve for 200 rounds
[200]	training's auc: 1	valid_1's auc: 1
[400]	training's auc: 1	valid_1's auc: 1
Early stopping, best iteration is:
[346]	training's auc: 1	valid_1's auc: 1
[('Fwd_Seg_Size_Min', 8461805.634903902), ('Dst_Port', 391764.42659282684), ('Init_Fwd_Win_Byts', 27185.396506385878), ('FIN_Flag_Cnt', 26925.513228621334), ('Fwd_Pkts/s', 13967.988023301587), ('Tot_Bwd_Pkts', 9536.47082862258), ('Flow_IAT_Max', 7784.71881014714), ('Fwd_IAT_Min', 5465.345918058418), ('Fwd_Header_Len', 3928.5121452212334), ('ACK_Flag_Cnt', 3208.563310623169), ('Flow_Pkts/s', 2240.7554825767875), ('Flow_Duration', 1825.6439771950245), ('Flow_IAT_Min', 1718.6581316078082), ('Fwd_IAT_Mean', 1418.941704876721), ('Idle_Mean', 1315.8299560546875), ('Bwd_Pkts/s', 1184.4577124938369), ('Fwd_IAT_Max

...

[LightGBM] [Warning] num_threads is set with nthread=28, will be overridden by n_jobs=24. Current value: num_threads=24
[LightGBM] [Warning] Unknown parameter: silent
Training until validation scores don't improve for 200 rounds
[200]	training's auc: 1	valid_1's auc: 1
[400]	training's auc: 1	valid_1's auc: 1
[600]	training's auc: 1	valid_1's auc: 1
Early stopping, best iteration is:
[562]	training's auc: 1	valid_1's auc: 1
[('Fwd_Seg_Size_Min', 8460805.519478604), ('Dst_Port', 388839.10177731514), ('FIN_Flag_Cnt', 35219.70318842679), ('Init_Fwd_Win_Byts', 24239.30231564492), ('Tot_Bwd_Pkts', 9605.855991363525), ('Flow_Pkts/s', 9558.54339779634), ('Flow_IAT_Max', 6091.612125873566), ('Fwd_Pkts/s', 5239.768989902543), ('Fwd_IAT_Max', 4613.515441972733), ('Fwd_Header_Len', 3569.291856497526), ('ACK_Flag_Cnt', 2465.210240781307), ('Flow_Duration', 2107.2811128841713), ('Bwd_Pkts/s', 1789.2674314156175), ('Flow_IAT_Min', 1573.058029350359), ('Fwd_IAT_Mean', 1510.4381721541286), ('Idle_Mean

D:\Users\Forsaken\anaconda3\envs\ep2\lib\site-packages\lightgbm\basic.py:1224: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  .format(key))


[LightGBM] [Warning] num_threads is set with nthread=28, will be overridden by n_jobs=24. Current value: num_threads=24
[LightGBM] [Warning] Unknown parameter: silent
Training until validation scores don't improve for 200 rounds
[200]	training's auc: 1	valid_1's auc: 1
[400]	training's auc: 1	valid_1's auc: 1
Early stopping, best iteration is:
[392]	training's auc: 1	valid_1's auc: 1
[('Fwd_Seg_Size_Min', 8460910.310569637), ('Dst_Port', 392232.07738757133), ('Init_Fwd_Win_Byts', 26963.470335911494), ('FIN_Flag_Cnt', 26501.227143459022), ('Flow_Duration', 11620.596598402597), ('Tot_Bwd_Pkts', 9349.07152428478), ('Flow_IAT_Max', 6591.242185179144), ('Fwd_Pkts/s', 5875.465637154528), ('Fwd_IAT_Min', 5673.433598751901), ('Fwd_Header_Len', 3666.0628982782364), ('ACK_Flag_Cnt', 3530.226303547621), ('Fwd_IAT_Max', 1669.7098157070577), ('Flow_IAT_Min', 1620.0865088094724), ('Fwd_IAT_Mean', 1575.2975581362844), ('Idle_Mean', 1346.52001953125), ('Flow_Pkts/s', 1169.3864311611687), ('Init_Bwd_Wi

D:\Users\Forsaken\anaconda3\envs\ep2\lib\site-packages\lightgbm\basic.py:1224: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  .format(key))


[LightGBM] [Warning] num_threads is set with nthread=28, will be overridden by n_jobs=24. Current value: num_threads=24
[LightGBM] [Warning] Unknown parameter: silent
Training until validation scores don't improve for 200 rounds
[200]	training's auc: 1	valid_1's auc: 1
[400]	training's auc: 1	valid_1's auc: 1
Early stopping, best iteration is:
[328]	training's auc: 1	valid_1's auc: 1
[('Fwd_Seg_Size_Min', 8463449.77152279), ('Dst_Port', 389348.59162569046), ('Init_Fwd_Win_Byts', 29044.190708067268), ('FIN_Flag_Cnt', 27361.991494327784), ('Flow_Duration', 11691.674647673965), ('Tot_Bwd_Pkts', 9614.828402370214), ('Flow_IAT_Max', 8613.116939669475), ('Fwd_Pkts/s', 6617.477288267575), ('Fwd_IAT_Max', 4699.037744613364), ('Fwd_Header_Len', 3583.334821008146), ('ACK_Flag_Cnt', 2827.8072197437286), ('Flow_IAT_Min', 1775.9475804679096), ('Fwd_IAT_Mean', 1581.4627877972962), ('Idle_Max', 1306.0999755859375), ('Fwd_IAT_Min', 1235.4989165570587), ('Flow_Pkts/s', 1217.2356489431113), ('Init_Bwd_W

In [52]:
xgb_trained_model, xgb_train = xgb_model(x_train, y_train)

************************************ 1/5 ************************************


XGBoostError: [14:56:02] C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/data/data.cc:1163: Check failed: valid: Input data contains `inf` or `nan`